### ------------------------------------------<br>Proyecto: Aire Analytics <br>Fecha última modificación: 2018-08-12 <br>Código: Tablas para el modelo LSTM<br> ------------------------------------------

 Los valores de calidad del aire ya están filtrados según estos contaminantes:
 - 01 -- Dióxido de Azufre (SO2)
 - 08 -- Dióxido de Nitrógeno (NO2)
 - 10 -- Partículas < 10 μm (PM10)
 - 14 -- Ozono (O3)

In [10]:
#vemos los datos de calidad del aire y eliminamos valores NA's
import pandas as pd
calidad = pd.read_csv('../dat/datos_modelo/datosCalidadfiltradosDiario.csv', sep=";", header=0, index_col=0)
print (calidad.index.size)
calidad = calidad[['fecha','ESTACION','MAGNITUD','valor']]
#eliminamos NA's si los hubiera
calidad=calidad.dropna(how='any') 
#cambiamos el tipo numérico de la estación a string
calidad['ESTACION'] = calidad['ESTACION'].astype(str)
calidad.ESTACION.dtype
print (calidad.index.size)
calidad.head()


171380
171380


,fecha,ESTACION,MAGNITUD,valor
1,2010-01-01,28079003,1,9.547083
2,2010-01-02,28079003,1,11.227500
3,2010-01-03,28079003,1,13.527083
4,2010-01-04,28079003,1,12.767917
5,2010-01-05,28079003,1,13.317500


In [13]:
#vemos los datos de climatológicos de AEMET y eliminamos valores NA's
aemetMadrid = pd.read_csv('../dat/datos_modelo/aemetMadrid.csv', sep=",", header=0, index_col=0)
#subset de las columnas de interés
aemetMadrid = aemetMadrid[['altitud','dir','fecha','horaracha','horatmax','horatmin','indicativo','prec','presMax','presMin','racha']]
print(aemetMadrid.index.size)
aemetMadrid=aemetMadrid.dropna(how='any')
print(aemetMadrid.index.size)
aemetMadrid.head()

39137
28865


,altitud,dir,fecha,horaracha,horatmax,horatmin,indicativo,prec,presMax,presMin,racha
2973,1030,99.0,2010-01-13,00:10,04:00,05:20,3110C,"3,9","892,7","879,5","11,9"
2975,1030,30.0,2010-01-15,00:10,14:30,08:00,3110C,"0,6","901,7","894,2","15,6"
2976,1030,99.0,2010-01-16,Varias,15:00,01:40,3110C,"0,5","901,7","898,7","2,8"
2977,1030,5.0,2010-01-17,16:10,16:00,Varias,3110C,"0,8","901,4","899,5","2,8"
2978,1030,26.0,2010-01-18,09:30,15:30,08:00,3110C,"0,0","900,3","895,3","2,5"


In [14]:
#leemos los valores de tráfico y eliminamos NA's
import pandas as pd
trafico = pd.read_csv('../dat/datos_modelo/traficoUnificado.csv', sep=";", header=0, index_col=0)
print (trafico.index.size)
trafico=trafico.dropna(how='any')
print(trafico.intensidad.dtype)
trafico.head()

398044
int64


,fecha,intensidad
id,,
1001,2014-01-01,744
1001,2014-01-02,1560
1001,2014-01-03,2847
1001,2014-01-04,1532
1001,2014-01-05,1548


In [15]:
#agrupamos los valores de tráfico por días
import pandas as pd
trafico['intensidad'] = pd.to_numeric(trafico['intensidad'])
#la columna intensidad es de tipo object-> la pasamos a numérica
trafico=trafico.groupby('fecha')[['intensidad']].sum()
print(trafico.index.size)
trafico.head()

1752


,intensidad
fecha,
2013-07-01,78715
2013-07-02,76204
2013-07-03,77972
2013-07-04,78756
2013-07-05,77442


In [16]:
estaciones_AEMET=['3110C', '3191E', '3200', '3129', '3196', '3195', '3266A', '2462','3338', '3111D', '3175']
estaciones_calidad=['28079003', '28079004', '28079005', '28079008', '28079010', '28079011', '28079013', '28079016',
 '28079017', '28079018', '28079020', '28079024', '28079026', '28079027','28079047','28079049',
 '28079054','28079055','28079056','28079057','28079058','28079059','28079086','28079099',
 '28079050','28079048','28079035','28079036','28079038','28079039','28079040','28079060']
#los contaminantes del tipo 'int64'
contaminantes=[1,  8, 14,  9, 10]

In [17]:
for estacion in estaciones_calidad:
    for contaminante in contaminantes:
        tabla_LSTM=calidad[(calidad['MAGNITUD']==contaminante) & (calidad['ESTACION']==estacion)]
        #print('filtramos por contaminante ' +str(contaminante)+' y estación '+str(estacion)) 
        #unimos el tráfico
        tabla_LSTM=pd.merge(tabla_LSTM, trafico, how='outer', on='fecha')
        #unimos datos AEMET
        tabla_LSTM=pd.merge(tabla_LSTM, aemetMadrid, how='outer', on='fecha')
        #eliminamos NA's 
        tabla_LSTM=tabla_cont.dropna(how='any') 
        #----- aquí habría que aplicar el modelo LSTM a cada tabla (1 por estación de calidad y contaminante)
        #----- o guardar las tablas generadas y más tarde aplicar el modelo LSTM una por una.
print(tabla_LSTM.index.size)  
tabla_LSTM.head()    

13096


,fecha,ESTACION,MAGNITUD,valor,intensidad,altitud,dir,horaracha,horatmax,horatmin,indicativo,prec,presMax,presMin,racha
7123,2013-09-03,28079060,10.0,25.541667,72450.0,1030.0,12.0,09:50,15:40,05:20,3110C,"0,0","927,7","909,4","10,0"
7124,2013-09-03,28079060,10.0,25.541667,72450.0,1004.0,16.0,09:30,15:10,06:20,3191E,"0,0","910,8","908,0","6,7"
7125,2013-09-03,28079060,10.0,25.541667,72450.0,620.0,21.0,12:30,14:45,05:40,3200,"0,0","952,5","948,2","6,1"
7126,2013-09-03,28079060,10.0,25.541667,72450.0,609.0,20.0,12:57,15:19,05:50,3129,"0,0","956,5","951,9","11,7"
7127,2013-09-03,28079060,10.0,25.541667,72450.0,690.0,21.0,14:00,15:15,06:00,3196,"0,0","944,9","940,7","7,2"
